<h3>Brochure for a company to be used for prospective clients, investors and potential recruits.</h3>

In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [6]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [7]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [8]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [9]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [10]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [11]:
create_brochure("Shein", "https://m.shein.com/eur/?lang=eur")

Found links: {'links': []}


# 🌟 Welcome to Shein: The Fashion Wonderland! 🌟

## Where Style Meets Savings 🎉 

Are you tired of wasting your precious time in crowded malls? Or sifting through endless racks of *meh* clothes? Look no further! Shein is your online haven for all things fabulous—at prices so good you might think we’re playing dress-up in a secret discount vault!

---

## Fashion for Everyone! 👗👖

At Shein, we believe that everyone deserves to look chic without breaking the bank. Whether you’re shopping for women, kids, men, or even searching that elusive Curve piece that flatters all types, we have it all! And if an item goes out of style, don’t worry—we have a sale for that! 🎊

- **Women**: From sassy tops to glamorous dresses.  
- **Men**: Because fashion isn't just for ladies (although, they do it best).  
- **Kids**: Tiny Tots = Big Style!  
- **Home & Living**: Turn your space into Instagram’s #1 contender! 🏠

---

## 🚢 Shipping and Returns: Easier Than Your Morning Coffee! ☕

- **Free Returns** within 30 days! Seriously! If it’s not love at first sight, send it back (but we genuinely hope it is).  
- You can even snag an **E-Gift Card** because who doesn’t love receiving cash in a virtual envelope? 🎁 

---

## Employee Culture: The ‘Shein’ State of Mind! 💼

We believe in fashion, fun, and a sprinkle of madness! Join us, and you’ll find a culture built on creativity, collaboration, and coffee (lots of coffee!). From 'Taco Tuesdays' to spontaneous dance-offs, our office feels like a fun party—one with trendy outfits, of course! 

### We’re Hiring! 🌟

- Looking for parts in our storytellers, trendsetters, and outfit connoisseurs crew?
- Fabulous opportunities await you! Come unleash your inner fashionista.

---

## Join the Shein Family! 🥳

Why shop somewhere else? At Shein, we craft an experience fit for a runway without a runway budget. Swipe right on that **FREE TRIAL** thing! (Wait, are we still on dating apps? 🤔)

Grab all this while enjoying our FLASH SALES that practically scream "Take my money!" Plus, with our app, you're always just one click away from your next wardrobe update. 

## Ready to Turn Heads? 🔄

So what are you waiting for? Dive into the world of Shein—where haute couture meets ‘haute can’t believe how cheap this is!’ 

---

Come for the fashion, stay for the savings, and who knows? You might just become a ‘Shein-stant’ fashion icon! 💃✨

*Disclaimer: Side effects may include an overflowing closet, spontaneous fashion shows in your living room, and an overwhelming number of selfies.* 🤳🎉

--- 

# Shop Now at [Shein](https://www.shein.com)! 

 <h2>Typewriter animation</h2>

In [12]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [13]:
stream_brochure("Shein", "https://m.shein.com/eur/?lang=eur")

Found links: {'links': []}



# Welcome to SHEIN: Where Your Closet Dreams Come True!

## Who Are We? 🤔
SHEIN isn’t just another fashion retailer; we’re your ticket to an endless wardrobe adventure. Imagine a world where your closet is as vibrant as your personality—well, that world is just a click away! Whether you're a trendsetting queen, a comfy chic enthusiast, or a shoe fanatic, we’ve got your back (and your front, and your sides). 

## What Do We Offer? 📦
- **Endless Fashion Choices:** From Curve, Home, to kids and beyond, we’ve got something for everyone in the family. Dress up your little ones, get stylish for yourselves, and don’t forget your pet—just kidding, we don’t sell pet clothes (yet).
- **Massive Sales Galore:** Who doesn’t love a good bargain? Our **Flash Sale** will have you saving while you splurge! 
- **Free Returns:** Not happy with your purchase? No worries! Just send it back within 30 days. (But we know you’ll love everything—you have good taste!)

## Customers Like You 💃
SHEIN caters to the fashion-loving folks of the world. Whether you're a *"Shopaholic McFashionista"* or someone who's just trying to stay trendy while binging on Netflix, we’ve got styles as diverse as our customer base. Straight from the runway to your wardrobe, we serve looks that even your friends will envy—or at least comment on!

## A Culture of Style 🕺
At SHEIN, we believe fashion should be as inclusive and diverse as a box of assorted chocolates (the good kind, not the ones you hide in the back of the pantry). Our team is a creative bunch who live, breathe, and kind of eat fashion (don’t worry, we brush our teeth). 

## Careers with SHEIN 🚀
Join us on this fierce fashion journey! We’re always on the lookout for passionate individuals who want to bring style into the lives of others. Whether you have an eye for design, a knack for marketing, or just know how to rock a pair of shoes, we want to hear from you! Here’s a tip: don’t forget that fabulous outfit; we love a good entrance!

## Why Choose SHEIN? 🌟
- **Fashion that doesn’t break the bank:** Enjoy affordable styles that won’t make your wallet weep.
- **Accessibility:** Fashion is for all! Whether you're tall, short, wide, or slim, we’ve got what you need. 
- **The SHEIN Family:** Join our community and enjoy the perks, discounts, and occasional surprise freebie!

So what are you waiting for? Dive into the world of SHEIN, fill your cart, and let your fashion journey begin! 

---

*Disclaimer: No pets were harmed in the making of this brochure. Friendly cats may approve of the fabric quality.* 🚫🐱
